<a href="https://colab.research.google.com/github/robert-myers/myanimelist-recommender/blob/master/notebooks/SVD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install surprise

In [0]:
import numpy as np
import pandas as pd
import random

from surprise import accuracy, Dataset, SVD, SVDpp, NMF, Reader
from surprise.dump import dump
from surprise.model_selection import cross_validate, GridSearchCV, train_test_split

my_seed = 8182868
random.seed(my_seed)
np.random.seed(my_seed)

In [0]:
%%time
# Creation of the dataframe.
df = pd.read_csv("https://s3.us-east-2.amazonaws.com/my.anime.list.sagemaker/surprise/uid_iid_rating.zip")

CPU times: user 13.8 s, sys: 1.61 s, total: 15.4 s
Wall time: 17.3 s


In [0]:
%%time
# A reader is needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 10))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['username', 'anime_id', 'my_score']], reader)

# sample random trainset and testset
# test set is made of 1% of the ratings.
trainset, testset = train_test_split(data, train_size=.99, test_size=.01)

CPU times: user 3min 16s, sys: 6.32 s, total: 3min 22s
Wall time: 3min 22s


In [0]:
%%time
algo = SVD()
predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions)

RMSE: 1.1859
CPU times: user 42min 4s, sys: 481 ms, total: 42min 5s
Wall time: 42min 6s


In [11]:
accuracy.rmse(predictions), accuracy.mae(predictions);

RMSE: 1.1859
MAE:  0.8652
